# Same approach as in CNNs but using logistic regression from pyspark library

In [1]:
import findspark

findspark.init()
 

In [2]:
from pyspark import SparkContext

In [3]:
sc = SparkContext(master="local[2]")

C:\Users\gprak\anaconda3\envs\my36v2\lib\site-packages\pyspark\context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [4]:
import pyspark

from pyspark.sql import SparkSession

# spark = SparkSession.builder.getOrCreate()

# df = spark.sql("select 'spark' as hello ")

# df.show()

In [5]:
#sc = spark

In [6]:
sc

<SparkContext master=local[2] appName=pyspark-shell>

In [7]:
spark = SparkSession.builder.appName("TextClassifierApp").getOrCreate()

In [8]:

df = spark.read.csv('../Zycus/86b13d9a4b8e11ec/project/training_data.csv',header=True,inferSchema=True)


In [9]:
df.show()

+--------------------+--------------------+
|               title|            category|
+--------------------+--------------------+
|    The Three Amigos|                None|
|Home Essentials B...|      Home & Kitchen|
|Cooper Wiring Qui...|Tools & Home Impr...|
|Baseboarders&reg;...|Tools & Home Impr...|
|The Great Wave Of...|     Office Products|
|Nemcor Pittsburgh...|      Home & Kitchen|
|Patrician Berkley...|                None|
|SouvNear 81461402...|                None|
|20 Qty. Halco 50W...|Tools & Home Impr...|
|      Rilakkuma Bowl|                None|
|Redbirdlinen 1pc ...|      Home & Kitchen|
|Hospital Bath Tow...|      Home & Kitchen|
|Symphony in Red a...|      Home & Kitchen|
|Big Train BLENDED...|Grocery & Gourmet...|
|Surpahs Round 11 ...|      Home & Kitchen|
|Mikasa Love Story...|                None|
|180 Snacks Nutty ...|                None|
|Anti-Slip Handle ...|      Home & Kitchen|
|Imagine Thicket G...|                None|
|KOHLER K-3754-96 ...|          

In [10]:
df.columns

['title', 'category']

In [11]:
#df=df.dropna()

In [12]:
df.columns

['title', 'category']

In [13]:
import pyspark.ml.feature

In [14]:
dir(pyspark.ml.feature)

['Binarizer',
 'BucketedRandomProjectionLSH',
 'BucketedRandomProjectionLSHModel',
 'Bucketizer',
 'ChiSqSelector',
 'ChiSqSelectorModel',
 'CountVectorizer',
 'CountVectorizerModel',
 'DCT',
 'ElementwiseProduct',
 'FeatureHasher',
 'HasFeaturesCol',
 'HasHandleInvalid',
 'HasInputCol',
 'HasInputCols',
 'HasLabelCol',
 'HasMaxIter',
 'HasNumFeatures',
 'HasOutputCol',
 'HasOutputCols',
 'HasRelativeError',
 'HasSeed',
 'HasStepSize',
 'HasThreshold',
 'HasThresholds',
 'HashingTF',
 'IDF',
 'IDFModel',
 'Imputer',
 'ImputerModel',
 'IndexToString',
 'Interaction',
 'JavaEstimator',
 'JavaMLReadable',
 'JavaMLWritable',
 'JavaModel',
 'JavaParams',
 'JavaTransformer',
 'MaxAbsScaler',
 'MaxAbsScalerModel',
 'MinHashLSH',
 'MinHashLSHModel',
 'MinMaxScaler',
 'MinMaxScalerModel',
 'NGram',
 'Normalizer',
 'OneHotEncoder',
 'OneHotEncoderModel',
 'PCA',
 'PCAModel',
 'Param',
 'Params',
 'PolynomialExpansion',
 'QuantileDiscretizer',
 'RFormula',
 'RFormulaModel',
 'RegexTokenizer',
 'R

In [15]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF

In [16]:
from pyspark.ml.feature import StringIndexer

# Filtering out the None categories to remove imbalance in dataset

In [17]:

# define a list of scores
l = ['None']

# filter out records by scores by list l
df = df.filter(~df.category.isin(l))

In [19]:
df.show()

+--------------------+--------------------+
|               title|            category|
+--------------------+--------------------+
|Home Essentials B...|      Home & Kitchen|
|Cooper Wiring Qui...|Tools & Home Impr...|
|Baseboarders&reg;...|Tools & Home Impr...|
|The Great Wave Of...|     Office Products|
|Nemcor Pittsburgh...|      Home & Kitchen|
|20 Qty. Halco 50W...|Tools & Home Impr...|
|Redbirdlinen 1pc ...|      Home & Kitchen|
|Hospital Bath Tow...|      Home & Kitchen|
|Symphony in Red a...|      Home & Kitchen|
|Big Train BLENDED...|Grocery & Gourmet...|
|Surpahs Round 11 ...|      Home & Kitchen|
|Anti-Slip Handle ...|      Home & Kitchen|
|2 Hannah Montana ...|      Home & Kitchen|
|Master Lock 1KALJ...|Tools & Home Impr...|
|Bauxko 9&quot; x ...|     Office Products|
|Nantucket Wine Bo...|      Home & Kitchen|
|French Laveder Bu...|Grocery & Gourmet...|
|Premier Faucet 11...|Tools & Home Impr...|
|4 Locking Swivel ...|Tools & Home Impr...|
|Filigree Heart Si...|      Home

In [20]:
tokenizer = Tokenizer(inputCol='title',outputCol='mytokens')
stopwords_remover = StopWordsRemover(inputCol='mytokens',outputCol='filtered_tokens')
vectorizer = CountVectorizer(inputCol='filtered_tokens',outputCol='rawFeatures')
idf = IDF(inputCol='rawFeatures',outputCol='vectorizedFeatures')

In [21]:
labelEncoder = StringIndexer(inputCol='category',outputCol='label').fit(df)

In [22]:
labelEncoder.transform(df).show(5)

+--------------------+--------------------+-----+
|               title|            category|label|
+--------------------+--------------------+-----+
|Home Essentials B...|      Home & Kitchen|  0.0|
|Cooper Wiring Qui...|Tools & Home Impr...|  1.0|
|Baseboarders&reg;...|Tools & Home Impr...|  1.0|
|The Great Wave Of...|     Office Products|  2.0|
|Nemcor Pittsburgh...|      Home & Kitchen|  0.0|
+--------------------+--------------------+-----+
only showing top 5 rows



In [24]:
df = labelEncoder.transform(df)

In [25]:
(trainDF,testDF) = df.randomSplit((0.7,0.3),seed=42)

In [26]:
from pyspark.ml.classification import LogisticRegression

In [27]:
lr = LogisticRegression(featuresCol='vectorizedFeatures',labelCol='label')

In [28]:
from pyspark.ml import Pipeline 

In [29]:
pipeline = Pipeline(stages=[tokenizer,stopwords_remover,vectorizer,idf,lr])

In [30]:
lr_model = pipeline.fit(trainDF)

In [31]:
predictions = lr_model.transform(testDF)

In [32]:
predictions.show()

+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|               title|            category|label|            mytokens|     filtered_tokens|         rawFeatures|  vectorizedFeatures|       rawPrediction|         probability|prediction|
+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| BAZIC Sports Pus...|     Office Products|  2.0|[, bazic, sports,...|[, bazic, sports,...|(22831,[13,37,46,...|(22831,[13,37,46,...|[3.00266660534800...|[0.00905206155373...|       2.0|
| Biedermann &amp;...|      Home & Kitchen|  0.0|[, biedermann, &a...|[, biedermann, &a...|(22831,[5,28,36,4...|(22831,[5,28,36,4...|[11.3768478904634...|[0.99964504021620...|       0.0|
| Bosco Chocolate ...|Grocery & Gourmet...|  3.0|[, bosco, chocol

In [33]:
predictions.columns

['title',
 'category',
 'label',
 'mytokens',
 'filtered_tokens',
 'rawFeatures',
 'vectorizedFeatures',
 'rawPrediction',
 'probability',
 'prediction']

In [34]:
predictions.select('rawPrediction','probability','category','label','prediction').show(10)

+--------------------+--------------------+--------------------+-----+----------+
|       rawPrediction|         probability|            category|label|prediction|
+--------------------+--------------------+--------------------+-----+----------+
|[3.00266660534800...|[0.00905206155373...|     Office Products|  2.0|       2.0|
|[11.3768478904634...|[0.99964504021620...|      Home & Kitchen|  0.0|       0.0|
|[5.40308521941286...|[0.02470772367624...|Grocery & Gourmet...|  3.0|       3.0|
|[-9.1145148190779...|[2.43142298934266...|Grocery & Gourmet...|  3.0|       3.0|
|[18.1281522066342...|[0.99997421919966...|      Home & Kitchen|  0.0|       0.0|
|[17.8383600083757...|[0.99999980806063...|      Home & Kitchen|  0.0|       0.0|
|[12.4767696506006...|[0.99979929245640...|      Home & Kitchen|  0.0|       0.0|
|[19.1838439372444...|[0.99999998528533...|      Home & Kitchen|  0.0|       0.0|
|[5.24114906356274...|[6.99694579856524...|     Office Products|  2.0|       2.0|
|[18.16365265171

In [35]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [36]:
evaluator = MulticlassClassificationEvaluator(labelCol='label',predictionCol='prediction',metricName='accuracy')

In [37]:
accuracy = evaluator.evaluate(predictions)

In [38]:
accuracy

0.7672266026739801

# Doing validation on unseen data

In [39]:

df2 = spark.read.csv('../Zycus/86b13d9a4b8e11ec/project/testing_data.csv',header=True,inferSchema=True)


In [40]:
validations = lr_model.transform(df2)

In [41]:
validations.show()

+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|               title|category|            mytokens|     filtered_tokens|         rawFeatures|  vectorizedFeatures|       rawPrediction|         probability|prediction|
+--------------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|Pom Pom Hair Band...|    null|[pom, pom, hair, ...|[pom, pom, hair, ...|(22831,[0,7,10,13...|(22831,[0,7,10,13...|[13.4053680494903...|[0.99998376788913...|       0.0|
|Mariposa Golf Bal...|    null|[mariposa, golf, ...|[mariposa, golf, ...|(22831,[213,1096,...|(22831,[213,1096,...|[7.24962069745842...|[0.80013346185125...|       0.0|
|Mediterranean Sna...|    null|[mediterranean, s...|[mediterranean, s...|(22831,[224,988,1...|(22831,[224,988,1...|[3.63146791470849...|[8.60805256092637..